<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Daniel Quilodrán Acuña
- Nombre de alumno 2: Katherine Rutte Poblete


### **Link de repositorio de GitHub:** `https://github.com/quilo98/Labs_MDS`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [96]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime as dt
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [97]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [98]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [99]:
df_retail.isna().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [115]:
def custom_features(data):
    # Ultima fecha
    t1 = dt.timedelta(days=1)
    ult = data["InvoiceDate"].max() + t1
    # Definimos un diccionario de agregaciones personalizadas
   # Definimos un diccionario de agregaciones personalizadas
    custom_aggregations = {
        'InvoiceDate': [
            ('min', 'min'),  # Primera compra
            ('max', 'max'),  # Última compra
            ('nunique', 'nunique'),  # Frequency
            ('Length', lambda x: (x.max() - x.min()).days),  # Length
            ('Recency', lambda x: (ult - x.max()).days)  # Recency
        ],
        'Price': [
            ('TotalSpent', lambda x: (x * data.loc[x.index, 'Quantity']).sum()),  # Total gastado
            ('MeanSpentPerVisit', lambda x: (x * data.loc[x.index, 'Quantity']).sum() / data.loc[x.index, 'InvoiceDate'].nunique())  # Promedio gastado por visita
        ]
    }

    # Realizamos las agregaciones definidas
    result = data.groupby('Customer ID').agg(custom_aggregations)

    # Renombramos las columnas para que tengan nombres más descriptivos
    result.columns = [
        'FirstPurchase', 'LastPurchase', 'Frequency', 'Length', 'Recency',
        'TotalSpent', 'Monetary'
    ]

    # Calcular periodicity (P) para cada cliente
    customer_data = data.sort_values(by=['Customer ID', 'InvoiceDate'])
    customer_data['IVT'] = customer_data.groupby('Customer ID')['InvoiceDate'].diff().dt.days
    customer_periodicity = customer_data.groupby('Customer ID')['IVT'].std()
    
    # Agregamos la columna 'Periodicity' al DataFrame resultante
    result['Periodicity'] = customer_periodicity
    result = result.fillna(0)
    return result

In [117]:
display(custom_features(df_retail)) 
df = custom_features(df_retail)

,FirstPurchase,LastPurchase,Frequency,Length,Recency,TotalSpent,Monetary,Periodicity
Customer ID,,,,,,,,
12346.0,2009-12-14 08:34:00,2010-06-28 13:53:00,11,196,165,372.86,33.896364,21.724076
12347.0,2010-10-31 14:20:00,2010-12-07 14:57:00,2,37,3,1323.32,661.660000,4.422346
12348.0,2010-09-27 14:59:00,2010-09-27 14:59:00,1,0,74,222.16,222.160000,0.000000
12349.0,2010-04-29 13:20:00,2010-10-28 08:23:00,3,181,43,2671.14,890.380000,16.200990
12351.0,2010-11-29 15:23:00,2010-11-29 15:23:00,1,0,11,300.93,300.930000,0.000000
...,...,...,...,...,...,...,...,...
18283.0,2010-02-19 17:16:00,2010-11-22 15:30:00,6,275,18,619.37,103.228333,11.783701
18284.0,2010-10-04 11:33:00,2010-10-04 11:33:00,1,0,67,461.68,461.680000,0.000000
18285.0,2010-02-17 10:24:00,2010-02-17 10:24:00,1,0,296,427.00,427.000000,0.000000


**Respúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [118]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Calculamos min(x) y max(x) para cada columna
        min_vals = X.min()
        max_vals = X.max()
        
        # Realizamos la transformación Min-Max para cada columna
        X_transformed = (X - min_vals) / (max_vals - min_vals)
        
        return X_transformed

In [119]:
transformer = ColumnTransformer(
    transformers=[('minmax_scaler', MinMax(), df.columns)],
    remainder='passthrough'
)

### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

In [120]:
df.isna().sum()

FirstPurchase    0
LastPurchase     0
Frequency        0
Length           0
Recency          0
TotalSpent       0
Monetary         0
Periodicity      0
dtype: int64

**Respuesta:**

In [121]:
def select_numeric_features(dataframe):
    return dataframe.select_dtypes(include=['number'])

# Define el pipeline
pipeline = Pipeline([
    ('custom_features', FunctionTransformer(func=custom_features, validate=False)),  # Mantén la transformación custom_features
    ('select_numeric', FunctionTransformer(func=select_numeric_features, validate=False)),  # Selecciona solo las columnas numéricas
    ('minmax_scaler', MinMax()),  # Aplica MinMaxScaler a las columnas numéricas
    ('tsne', TSNE(n_components=2))  # Aplica t-SNE a las columnas numéricas
])

# Aplicar el pipeline al DataFrame
X_reduced = pipeline.fit_transform(df_retail)

reduced_df = pd.DataFrame(data=X_reduced, columns=['Componente 1', 'Componente 2'])

# Graficar las componentes con Plotly Express
fig = px.scatter(reduced_df, x='Componente 1', y='Componente 2', title='Reducción de dimensionalidad utilizando T-SNE')
fig.update_xaxes(title_text='Componente 1')
fig.update_yaxes(title_text='Componente 2')
fig.show()

c:\Users\kathe\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

c:\Users\kathe\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [122]:
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px

# Función para calcular la inercia (suma de las distancias al cuadrado) de K-Means para un rango de k
def calculate_inertia(X, k_range):
    inertias = []
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(X)
        inertias.append(kmeans.inertia_)
    return inertias

# Rango de valores de k que vamos a probar
k_range = range(1, 21)

# Crear un nuevo pipeline para K-Means
kmeans_pipeline = Pipeline([
    ('custom_features', FunctionTransformer(func=custom_features, validate=False)),  # Mantén la transformación custom_features
    ('select_numeric', FunctionTransformer(func=select_numeric_features, validate=False)),  # Selecciona solo las columnas numéricas
    ('minmax_scaler', MinMax()),  # Aplica MinMaxScaler a las columnas numéricas
    ('kmeans', KMeans())
])

# Aplicar el pipeline al DataFrame y calcular la inercia para cada valor de k
inertias = calculate_inertia(kmeans_pipeline.fit_transform(df_retail), k_range)

# Crear un DataFrame para los datos de la inercia
inertia_df = pd.DataFrame({'k': k_range, 'inertia': inertias})

# Graficar la inercia en función de k con Plotly Express
fig = px.line(inertia_df, x='k', y='inertia', title='Método del Codo para K-Means')
fig.update_xaxes(title_text='Número de Clusters (k)')
fig.update_yaxes(title_text='Inercia')
fig.show()


#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [123]:
from sklearn.cluster import KMeans

# Crear un nuevo pipeline para K-Means con k=4
kmeans_pipeline = Pipeline([
    ('custom_features', FunctionTransformer(func=custom_features, validate=False)),  # Mantén la transformación custom_features
    ('select_numeric', FunctionTransformer(func=select_numeric_features, validate=False)),  # Selecciona solo las columnas numéricas
    ('minmax_scaler', MinMax()),  # Aplica MinMaxScaler a las columnas numéricas
    ('kmeans', KMeans(n_clusters=5, random_state=0))  # Utiliza k=5
])

# Ajustar el modelo K-Means al DataFrame
kmeans_pipeline.fit(df_retail)

# Obtener las etiquetas de cluster asignadas a cada registro
cluster_labels = kmeans_pipeline.named_steps['kmeans'].labels_

# Agregar las etiquetas de cluster al DataFrame original
df['Cluster'] = cluster_labels

# Calcular los promedios por cluster
cluster_means = df.groupby('Cluster').mean()

# Calcular la cantidad de clientes en cada cluster
cluster_counts = df['Cluster'].value_counts()

# Combinar promedios y cantidad de clientes en una sola tabla
cluster_summary = pd.concat([cluster_means, cluster_counts], axis=1)
cluster_summary.rename(columns={'Cluster': 'Cantidad_clientes'}, inplace=True)
# Imprimir la tabla de promedios y cantidad de clientes por cluster
display(cluster_summary)


,Frequency,Length,Recency,TotalSpent,Monetary,Periodicity,Cantidad_clientes
0,4.317729,188.020916,61.813745,1582.951724,367.584565,20.886145,1004
1,1.274892,5.701299,305.809524,512.688602,358.266465,1.172677,462
2,1.752356,17.807198,43.542416,691.295304,377.876301,2.454493,1167
3,10.376988,322.117867,23.135641,5389.530626,412.396817,20.165788,1069
4,1.825163,34.754902,187.653595,660.029771,335.815661,5.235518,612


In [124]:
cluster_means

,Frequency,Length,Recency,TotalSpent,Monetary,Periodicity
Cluster,,,,,,
0,4.317729,188.020916,61.813745,1582.951724,367.584565,20.886145
1,1.274892,5.701299,305.809524,512.688602,358.266465,1.172677
2,1.752356,17.807198,43.542416,691.295304,377.876301,2.454493
3,10.376988,322.117867,23.135641,5389.530626,412.396817,20.165788
4,1.825163,34.754902,187.653595,660.029771,335.815661,5.235518


``
¿Es posible observar agrupaciones coherentes?
``
Si es posibile observar agrupaciones coherentes, ya que si bien algunos clusters son similares en algunas carectiristicas, difieren completamente en las otras. Por ejemplo es coherente que aquellos clientes que realizaron su primera compra hace más tiempo, tengan una frecuencia de visita más alta que aquellos que no llevan siendo clientes tanto tiempo.

``
¿Qué tipo de clientes posee el retail?
``
-   **Cluster 1: Clientes Fieles** \
 Este cluster está compuesto por 1004 clientes que presentan características de lealtad destacadas. Con un alto valor en la característica "Length," sugiere que son clientes fieles y que han realizado su última compra hace aproximadamente dos meses. Estos clientes tienden a gastar un monto promedio de 367 USD por visita y han visitado la tienda con regularidad, lo que los convierte en una base sólida de ingresos continuos.

-   **Cluster 2 - Clientes Inactivos** \
En contraste, el Cluster 2 incluye 462 clientes que, aunque han realizado una o dos compras en el pasado, han perdido interés en la tienda, ya que su última compra ocurrió hace aproximadamente 9 meses. Su gasto promedio es de 358 USD. La tienda podría emprender estrategias de reactivación, como campañas de marketing específicas y descuentos personalizados, para atraer nuevamente su atención. Para mantenerlos comprometidos, la tienda podría ofrecer programas de lealtad y recompensas exclusivas.

-   **Cluster 3 - Clientes Recientes** \
Este grupo de 1167 clientes es relativamente nuevo, habiendo realizado una o dos compras en el último mes y 2 semanas aproximadamente. Su gasto promedio es de 377 USD. Son clientes en la etapa de adquisición. La tienda debe enfocarse en brindar experiencias positivas y ofertas atractivas para fomentar su lealtad a largo plazo.

-   **Cluster 4 - Clientes Fieles de Alto Gasto**:
Con 1069 clientes, este cluster destaca por su alta fidelidad y poder adquisitivo. Han realizado más de 10 compras y su última compra ocurrió hace aproximadamente 3 semanas. El gasto promedio es el más alto entre los clusters, alcanzando los 477 USD. Estos clientes son valiosos y podrían ser considerados como clientes VIP. La tienda debe mantener su satisfacción y considerar ofrecer productos exclusivos o servicios personalizados.

-   **Cluster 5 - Clientes Inactivos de Bajo Gasto**:
Por último, el Cluster 5 comprende 612 clientes que han realizado una o dos compras, pero hace aproximadamente 6 meses. Su gasto promedio es de 335 USD. Para reactivar su interés, la tienda podría implementar estrategias de marketing dirigidas y ofertas especiales.


La segmentación de clientes en el retail es esencial para adaptar estrategias de marketing y retención. Desde clientes fieles y valiosos hasta aquellos que necesitan ser reactivados, cada cluster requiere un enfoque específico. La tienda debe cuidar a sus clientes fieles, atraer a los recién llegados, y esforzarse por reactivar a los inactivos. La comprensión de estas segmentaciones es crucial para maximizar el valor del cliente y optimizar el ciclo de vida del cliente.

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [125]:
# Agregar la columna 'Cluster' al DataFrame reduced_df
reduced_df['Cluster'] = cluster_labels

# Graficar las componentes con colores basados en los clusters de K-Means
fig = px.scatter(reduced_df, x='Componente 1', y='Componente 2', color='Cluster', title='Reducción de dimensionalidad utilizando T-SNE con Clusters de K-Means')
fig.update_xaxes(title_text='Componente 1')
fig.update_yaxes(title_text='Componente 2')
fig.show()


``
¿Se separan bien los distintos clusters en la visualización?
``

A simple vista si se separan bien, hay algunos puntos donde es menos notoria la separación, pero en general los clusters logran distinguirse uno del otro.


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>